In [1]:

import os

from argparse import Namespace
import collections
import nltk.data
import numpy as np
import pandas as pd
import re
import string
from tqdm import tqdm_notebook

In [17]:
args=Namespace(
    raw_dataset_txt='../data/books/frankenstein.txt',
    window_size=5,
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv='../data/books/frankenstein_with_splits.csv',
    seed=1337
)

In [3]:
import nltk
nltk.download('punkt')

from nltk import word_tokenize,sent_tokenize

[nltk_data] Downloading package punkt to /home/hwm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
with open(args.raw_dataset_txt) as fp:
    book=fp.read()
sentences=sent_tokenize(book)

In [5]:
print (len(sentences), "sentences")
print ("Sample:", sentences[100])

3427 sentences
Sample: No incidents have hitherto befallen us that would make a figure in a
letter.


In [6]:
def preprocess_text(text):
    text=' '.join(word.lower() for word in text.split(' '))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

In [7]:
cleaned_sentences = [preprocess_text(sentence) for sentence in sentences]

In [8]:
# Global vars
MASK_TOKEN = "<MASK>"

In [9]:
flatten=lambda outer_list: [item for inner_list in outer_list for item in inner_list]
windows=flatten([list(nltk.ngrams([MASK_TOKEN]*args.window_size+sentence.split(' ')+[MASK_TOKEN]*args.window_size,args.window_size*2+1))
               for sentence in tqdm_notebook(cleaned_sentences)])

In [10]:
len(windows)

90698

In [11]:
windows[:3]

[('<MASK>',
  '<MASK>',
  '<MASK>',
  '<MASK>',
  '<MASK>',
  'frankenstein',
  ',',
  'or',
  'the',
  'modern',
  'prometheus'),
 ('<MASK>',
  '<MASK>',
  '<MASK>',
  '<MASK>',
  'frankenstein',
  ',',
  'or',
  'the',
  'modern',
  'prometheus',
  'by'),
 ('<MASK>',
  '<MASK>',
  '<MASK>',
  'frankenstein',
  ',',
  'or',
  'the',
  'modern',
  'prometheus',
  'by',
  'mary')]

In [13]:
data=[]
for window in tqdm_notebook(windows):
    target_token=window[args.window_size]
    context=[]
    for i,token in enumerate(window):
        #如果token 为MASK_TOKEN 或者为目标token 就不加入，
        if token==MASK_TOKEN or i ==args.window_size:
            continue
        else:
            context.append(token)
    data.append([' '.join(token for token in context), target_token])
cbow_data = pd.DataFrame(data, columns=["context", "target"])

In [14]:
n=len(cbow_data)
def get_split(row_num):
    if row_num <= n*args.train_proportion:
        return 'train'
    elif (row_num > n*args.train_proportion) and (row_num <= n*args.train_proportion + n*args.val_proportion):
        return 'val'
    else:
        return 'test'
cbow_data['split']= cbow_data.apply(lambda row: get_split(row.name), axis=1)

In [15]:
cbow_data.head()

,context,target,split
0,", or the modern prometheus",frankenstein,train
1,frankenstein or the modern prometheus by,",",train
2,"frankenstein , the modern prometheus by mary",or,train
3,"frankenstein , or modern prometheus by mary wo...",the,train
4,"frankenstein , or the prometheus by mary wolls...",modern,train


In [18]:
cbow_data.to_csv(args.output_munged_csv, index=False)